<a href="https://colab.research.google.com/github/vkt1414/Cloud-Resources-Workflows/blob/main/Notebooks/Totalsegmentator/postProcessingExtractPerframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Installing Packages (local)**

In [ ]:
# %%capture
# !sudo apt-get install lz4

In [ ]:
%%capture
!pip install pydicom google-cloud-bigquery pyarrow db_dtypes

###**Importing Packages**

In [ ]:
import os
import shutil
import pandas as pd
import pydicom
import traceback
import logging
from tqdm import tqdm
import subprocess  # Import the subprocess module

###**Parameters for papermill**

In [ ]:
segFilesCsv=''
jsonServiceAccountFile=''

In [ ]:
!gcloud auth activate-service-account --key-file={jsonServiceAccountFile}

In [ ]:
data= pd.read_csv(segFilesCsv)
data

In [ ]:
# # Add a new column called 'new_destination' to your DataFrame
# data['new_destination'] = ''

# # Populate the 'new_destination' column with values that are a concatenation of 'bucket_id' and 'batch_id'
# data['new_destination'] = data["dicomsegAndRadiomicsSR_CompressedFiles"] +' '+'gs://perframefunctionalgroupssequence/' +'batch_'+ data['entity:twoVM_2023_07_07_22_16_id'].astype(str)+'/'

# data


In [ ]:
# !gcloud auth login

In [ ]:
# !gcloud config set project 'idc-external-025'

In [ ]:
# urls= data['new_destination'].to_list()

In [ ]:
# for url in urls:
#   !gsutil cp {url}

In [ ]:
# Initialize logging
logging.basicConfig(filename="console_output.txt", level=logging.INFO)

# Create an output directory to store CSV or Parquet files
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)


seg_download_urls = data['new_destination'].to_list()
batch_count = 1  # Counter for batch folders

In [ ]:
# try:
#     for url in tqdm(seg_download_urls):
#         logging.info('Processing URL: %s', url)

#         # Clean up previous data if necessary
#         try:
#             shutil.rmtree('itkimage2segimage')
#             shutil.rmtree('decompressedSegmentationsDICOM')
#         except OSError:
#             pass

#         os.mkdir('decompressedSegmentationsDICOM')
#         # Download and process the data
#         try:
#             # Download and decompress data
#             !gsutil cp {url} . > /dev/null 2>&1
#             !lz4 -d --rm dicomsegAndRadiomicsSR_DICOMsegFiles.tar.lz4 -c | tar --strip-components=1 -xvf - > /dev/null 2>&1
#             !find ./itkimage2segimage -name '*.dcm.lz4' -exec mv -t decompressedSegmentationsDICOM {} + > /dev/null 2>&1
#             !lz4 -d -m --rm "decompressedSegmentationsDICOM"/*.lz4 > /dev/null 2>&1
#             print('files successfully decompressed')
#         except Exception as download_error:
#             logging.error('Error during download and decompression: %s', str(download_error))
#             continue  # Skip this batch and continue to the next

#         # # Create a batch folder
#         # batch_folder = os.path.join(output_dir, f'batch_{batch_count}')
#         # os.makedirs(batch_folder, exist_ok=True)

#         # Find all series IDs and add them to the DataFrame
#         series_ids = [filename.split('_')[0] for filename in os.listdir('decompressedSegmentationsDICOM')]
#         # Create a list to hold data from all series in the batch
#         batch_data = []

#         for series_id in series_ids:
#             pffgs = pydicom.dcmread(f'decompressedSegmentationsDICOM/{series_id}', specific_tags=["ReferencedSeriesSequence","PerFrameFunctionalGroupsSequence"], stop_before_pixels=True)
#             referencedSeriesInstanceUID = pffgs.ReferencedSeriesSequence[0].SeriesInstanceUID
#             data = []
#             # Extract data from Per-frame Functional Groups Sequence
#             if "PerFrameFunctionalGroupsSequence" in pffgs:
#                 for item in pffgs.PerFrameFunctionalGroupsSequence:
#                     frame_data = {
#                         'ReferencedSeriesSequence_SeriesInstanceUID': referencedSeriesInstanceUID,
#                         'FrameContentSequence_DimensionIndexValues': [str(s) for s in list(item.FrameContentSequence[0].DimensionIndexValues)],
#                         'PlanePositionSequence_ImagePositionPatient': [str(s) for s in list(item.PlanePositionSequence[0].ImagePositionPatient)],
#                         'SegmentIdentificationSequence_ReferencedSegmentNumber': item.SegmentIdentificationSequence[0].ReferencedSegmentNumber

#                     }

#                     # Extract attributes from Derivation Image Sequence
#                     derivation_image_sequence = item.DerivationImageSequence
#                     if derivation_image_sequence:
#                         source_image_sequence = derivation_image_sequence[0].SourceImageSequence
#                         if source_image_sequence:
#                             frame_data['DerivationImageSequence_SourceImageSequence_ReferencedSOPClassUID'] = source_image_sequence[0].ReferencedSOPClassUID
#                             frame_data['DerivationImageSequence_SourceImageSequence_ReferencedSOPInstanceUID'] = source_image_sequence[0].ReferencedSOPInstanceUID
#                             purpose_of_reference_code_sequence= source_image_sequence[0].PurposeOfReferenceCodeSequence
#                             if purpose_of_reference_code_sequence:
#                                 frame_data['DerivationImageSequence_SourceImageSequence_PurposeOfReferenceCodeSequence_CodeValue'] = purpose_of_reference_code_sequence[0].CodeValue
#                                 frame_data['DerivationImageSequence_SourceImageSequence_PurposeOfReferenceCodeSequence_CodingSchemeDesignator'] = purpose_of_reference_code_sequence[0].CodingSchemeDesignator
#                                 frame_data['DerivationImageSequence_SourceImageSequence_PurposeOfReferenceCodeSequence_CodeMeaning'] = purpose_of_reference_code_sequence[0].CodeMeaning
#                     # Extract attributes from Derivation Code Sequence
#                     derivation_code_sequence = derivation_image_sequence[0].DerivationCodeSequence
#                     if derivation_code_sequence:
#                         frame_data['DerivationImageSequence_DerivationCodeSequence_CodeValue'] = derivation_code_sequence[0].CodeValue
#                         frame_data['DerivationImageSequence_DerivationCodeSequence_CodingSchemeDesignator'] = derivation_code_sequence[0].CodingSchemeDesignator
#                         frame_data['DerivationImageSequence_DerivationCodeSequence_CodeMeaning'] = derivation_code_sequence[0].CodeMeaning

#                     data.append(frame_data)

#             # Add data from this series to the batch_data list
#             batch_data.extend(data)

#         # Create a DataFrame from the batch_data list
#         df = pd.DataFrame(batch_data)

#         # Generate a single CSV file for the batch
#         csv_filename = os.path.join(output_dir, f'batch_{batch_count}.csv')
#         df.to_csv(csv_filename, index=False)

#         # Compress the CSV file using lz4
#         #compressed_csv_filename = f'batch_{batch_count}.csv.lz4'
#         #subprocess.run(["lz4", csv_filename, compressed_csv_filename])

#         batch_count += 1
#         # perFrame_job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")
#         # perFrame_job = client.load_table_from_dataframe(df, "idc-external-025.terra.perFrameTest",job_config=perFrame_job_config)

# except Exception as e:
#     logging.error('An error occurred: %s', str(e))

# finally:
#     logging.info('Processing complete.')


In [ ]:
import os
import logging
import shutil
import pandas as pd
import pydicom
from tqdm import tqdm

# Initialize an empty list to store data from all URLs
combined_data = []

try:
    for url in tqdm(seg_download_urls):
        logging.info('Processing URL: %s', url)

        # Clean up previous data if necessary
        try:
            shutil.rmtree('itkimage2segimage')
            shutil.rmtree('decompressedSegmentationsDICOM')
        except OSError:
            pass

        os.mkdir('decompressedSegmentationsDICOM')
        # Download and process the data
        try:
            # Download and decompress data
            !gsutil cp {url} . > /dev/null 2>&1
            !lz4 -d --rm dicomsegAndRadiomicsSR_DICOMsegFiles.tar.lz4 -c | tar --strip-components=1 -xvf - > /dev/null 2>&1
            !find ./itkimage2segimage -name '*.dcm.lz4' -exec mv -t decompressedSegmentationsDICOM {} + > /dev/null 2>&1
            !lz4 -d -m --rm "decompressedSegmentationsDICOM"/*.lz4 > /dev/null 2>&1
            print('files successfully decompressed')
        except Exception as download_error:
            logging.error('Error during download and decompression: %s', str(download_error))
            continue  # Skip this URL and continue to the next

        # Find all series IDs and add them to the combined_data list
        series_ids = [filename.split('_')[0] for filename in os.listdir('decompressedSegmentationsDICOM')]

        for series_id in series_ids:
            pffgs = pydicom.dcmread(f'decompressedSegmentationsDICOM/{series_id}', specific_tags=["ReferencedSeriesSequence","PerFrameFunctionalGroupsSequence"], stop_before_pixels=True)
            referencedSeriesInstanceUID = pffgs.ReferencedSeriesSequence[0].SeriesInstanceUID
            data = []

            # Extract data from Per-frame Functional Groups Sequence
            if "PerFrameFunctionalGroupsSequence" in pffgs:
                for item in pffgs.PerFrameFunctionalGroupsSequence:
                    frame_data = {
                        'ReferencedSeriesSequence_SeriesInstanceUID': referencedSeriesInstanceUID,
                        'FrameContentSequence_DimensionIndexValues': [str(s) for s in list(item.FrameContentSequence[0].DimensionIndexValues)],
                        'PlanePositionSequence_ImagePositionPatient': [str(s) for s in list(item.PlanePositionSequence[0].ImagePositionPatient)],
                        'SegmentIdentificationSequence_ReferencedSegmentNumber': item.SegmentIdentificationSequence[0].ReferencedSegmentNumber
                    }

                    # Extract attributes from Derivation Image Sequence
                    derivation_image_sequence = item.DerivationImageSequence
                    if derivation_image_sequence:
                        source_image_sequence = derivation_image_sequence[0].SourceImageSequence
                        if source_image_sequence:
                            frame_data['DerivationImageSequence_SourceImageSequence_ReferencedSOPClassUID'] = source_image_sequence[0].ReferencedSOPClassUID
                            frame_data['DerivationImageSequence_SourceImageSequence_ReferencedSOPInstanceUID'] = source_image_sequence[0].ReferencedSOPInstanceUID
                            purpose_of_reference_code_sequence = source_image_sequence[0].PurposeOfReferenceCodeSequence
                            if purpose_of_reference_code_sequence:
                                frame_data['DerivationImageSequence_SourceImageSequence_PurposeOfReferenceCodeSequence_CodeValue'] = purpose_of_reference_code_sequence[0].CodeValue
                                frame_data['DerivationImageSequence_SourceImageSequence_PurposeOfReferenceCodeSequence_CodingSchemeDesignator'] = purpose_of_reference_code_sequence[0].CodingSchemeDesignator
                                frame_data['DerivationImageSequence_SourceImageSequence_PurposeOfReferenceCodeSequence_CodeMeaning'] = purpose_of_reference_code_sequence[0].CodeMeaning

                    # Extract attributes from Derivation Code Sequence
                    derivation_code_sequence = derivation_image_sequence[0].DerivationCodeSequence
                    if derivation_code_sequence:
                        frame_data['DerivationImageSequence_DerivationCodeSequence_CodeValue'] = derivation_code_sequence[0].CodeValue
                        frame_data['DerivationImageSequence_DerivationCodeSequence_CodingSchemeDesignator'] = derivation_code_sequence[0].CodingSchemeDesignator
                        frame_data['DerivationImageSequence_DerivationCodeSequence_CodeMeaning'] = derivation_code_sequence[0].CodeMeaning

                    data.append(frame_data)

            # Add data from this series to the combined_data list
            combined_data.extend(data)

except Exception as e:
    logging.error('An error occurred: %s', str(e))

finally:
    # Create a DataFrame from the combined_data list
    df = pd.DataFrame(combined_data)

    # Generate a single CSV file for all the data
    csv_filename = 'perFrameFunctionalGroupSequence.csv'
    df.to_csv(csv_filename, index=False)
    !lz4 --rm 'perFrameFunctionalGroupSequence.csv' 'perFrameFunctionalGroupSequence.csv.lz4'

    logging.info('Processing complete.')
